In [1]:
import sqlite3
from pxr import Usd, UsdGeom, Gf, Vt, Sdf

In [ ]:
def load_table(db_file, table, cn):
   conn = sqlite3.connect(db_file)
   conn.row_factory = sqlite3.Row
   cursor = conn.cursor()
   query = f'SELECT * FROM {table};'
   cursor.execute(query)
   results = cursor.fetchall()
   columns=results[0].keys()
   print(table,len(results), columns)
   res={}
   for r in results:      
      ins={}
      for c in columns[1:]:
         ins[c]=r[c]
      res[r['id']]=type(cn,(),ins)
   cursor.close()
   conn.close()
   return res

In [ ]:
database_file = "geometry_atlas.db"
sh_d = load_table(database_file, 'Shapes','SH')

In [ ]:
def Create_Box(Parameters):
    path = "assests/shapes/shape_"+ str(Parameters[0])+".usda"
    shape_name = "/shape_"+str(Parameters[0])
    
    stage = Usd.Stage.CreateNew(path)
    xform = UsdGeom.Xform.Define(stage, shape_name)
    box = UsdGeom.Cube.Define(stage, shape_name+"/geometry")

    key_value_pairs=Parameters[2].split(";")
    box_length=[]
    for value in key_value_pairs:
        box_length.append(float(value.split("=")[1])*2)
    extent = Vt.Vec3fArray(1, Gf.Vec3f(box_length[0],box_length[1],box_length[2]))
    box.GetPrim().CreateAttribute("extent", Sdf.ValueTypeNames.Float3Array).Set(extent)

    # Add the box as a child of the xform
    xform.GetPrim().GetChildren().append(box.GetPrim())

    # Save the USD file
    stage.GetRootLayer().Save()
    return

for i, sh in sh_d.items():
    print(i)
